In [ ]:
from astropy.io import ascii
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import mahalanobis as mdist
from scipy.stats import chi2
from tensorflow.keras.utils import to_categorical

In [ ]:
Train=np.genfromtxt('Training.csv',skip_header=1,delimiter=',')

In [ ]:
TrainX=Train[:,0:2]
TrainY=toTrain[:,2]
